In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
HYGyas = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HYG")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")
SPY_Idx = pd.read_excel(io="SP500 Index data.xlsx",sheet_name="SPY")

portfolio = portfolio.set_index("Date").sort_index()
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()
SPY_Idx = SPY_Idx.set_index("Date").sort_index()
HYGyas = HYGyas.set_index("Date").sort_index()

PortfCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","Total Value"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
PortfCorrData.loc[:,"Total Returns"] = PortfCorrData["Total Value"].pct_change()

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(HYyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS","INDEX_Z_SPREAD_BP":"HY_INDEX_Z_SPREAD"})
PortfCorrData.loc[:,"HY_Index_OAS_Diff"]=PortfCorrData["HY_Index_OAS"].diff()
PortfCorrData.loc[:,"HY_INDEX_Z_SPREAD_Diff"]=PortfCorrData["HY_INDEX_Z_SPREAD"].diff()

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
PortfCorrData.loc[:,"10yYieldDiff"] = PortfCorrData["10y_UST_INDEX_PX"].diff()

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_Spread","OAS_SOVEREIGN_CURVE":"IG_Index_OAS"})
PortfCorrData.loc[:,"IG_Index_Z_Spread_Diff"]=(PortfCorrData["IG_INDEX_Z_Spread"].diff())
PortfCorrData.loc[:,"IG_Index_OAS_Diff"]=PortfCorrData["IG_Index_OAS"].diff()

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(SPY_Idx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"PX_LAST":"SPX_LAST_PX"})
PortfCorrData.loc[:,"ReturnSPX_Idx"] = PortfCorrData["SPX_LAST_PX"].pct_change()

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(HYGyas.loc["2021-01-11":"2025-12-31","YAS_MOD_DUR"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"YAS_MOD_DUR":"HYG_MOD_DUR_YAS"})

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

PortfCorrData = PortfCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
PortfCorrData = PortfCorrData.rename(columns={"PX_LAST":"Vix Idx"})

PortfCorrData=PortfCorrData.dropna()
PortfCorrData = PortfCorrData.sort_index(ascending=False)

In [7]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(PortfCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPX_Idx","MOVE Idx","Vix Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPX_Idx","MOVE Idx","Vix Idx"]))

X_pca = pcav1.transform(X_Std)

PortfCorrData.loc[:,"PC1"]=X_pca[:,0]
PortfCorrData.loc[:,"PC2"]=X_pca[:,1]
PortfCorrData.loc[:,"PC3"]=X_pca[:,2]
PortfCorrData.loc[:,"PC4"]=X_pca[:,3]
PortfCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(PortfCorrData[["PC1","PC2","PC3","PC4"]])
Y = PortfCorrData["Total Returns"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.39707456 0.20850729 0.1692098  0.11470395 0.07174926 0.03875513]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_Z_Spread_Diff  ReturnSPX_Idx  \
0           0.581172     -0.163460                0.566444      -0.486361   
1          -0.175898      0.465330               -0.121508      -0.053439   
2          -0.002617      0.757975                0.009043      -0.430689   
3          -0.083715     -0.163153               -0.061582       0.135200   
4           0.182591      0.373773                0.554277       0.719507   
5           0.768729      0.126087               -0.594373       0.197826   

   MOVE Idx   Vix Idx  
0  0.113516  0.255398  
1  0.622952  0.588930  
2 -0.469168 -0.140622  
3 -0.614136  0.753086  
4 -0.041945 -0.016779  
5 -0.001792  0.027190  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Total Returns   R-squared:                       0.656
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     589.0
Date:                Fri, 16 Jan 2026   Prob (F-statistic):          2.29e-284
Time:                        19:57:41   Log-Likelihood:                 5603.3
No. Observations:                1240   AIC:                        -1.120e+04
Df Residuals:                    1235   BIC:                        -1.117e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.784e-05   7.51e-05     -0.504      0.614      -0.000       0.000
PC1        -4.125e-05   4.86e-05     -0.848      0.396      -0.000    5.42e-05
PC2           -0.0016   6.71e-05    -23.273      0.000      -0.002      -0.001
PC3           -0.0031   7.45e-05    -41.604      0.000      -0.003      -0.003
PC4            0.0008   9.05e-05      9.101      0.000       0.001       0.001
==============================================================================
Omnibus:                       59.110   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              177.833
Skew:                          -0.138   Prob(JB):                     2.42e-39
Kurtosis:                       4.835   Cond. No.                         1.86
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(PortfCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPX_Idx","MOVE Idx","Vix Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPX_Idx","MOVE Idx","Vix Idx"]))

X_pca = pcav1.transform(X_Std)

PortfCorrData.loc[:,"PC1"]=X_pca[:,0]
PortfCorrData.loc[:,"PC2"]=X_pca[:,1]
PortfCorrData.loc[:,"PC3"]=X_pca[:,2]
PortfCorrData.loc[:,"PC4"]=X_pca[:,3]
PortfCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(PortfCorrData[["PC2","PC3","PC4"]])
Y = PortfCorrData["Total Returns"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.39707456 0.20850729 0.1692098  0.11470395 0.07174926 0.03875513]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_Z_Spread_Diff  ReturnSPX_Idx  \
0           0.581172     -0.163460                0.566444      -0.486361   
1          -0.175898      0.465330               -0.121508      -0.053439   
2          -0.002617      0.757975                0.009043      -0.430689   
3          -0.083715     -0.163153               -0.061582       0.135200   
4           0.182591      0.373773                0.554277       0.719507   
5           0.768729      0.126087               -0.594373       0.197826   

   MOVE Idx   Vix Idx  
0  0.113516  0.255398  
1  0.622952  0.588930  
2 -0.469168 -0.140622  
3 -0.614136  0.753086  
4 -0.041945 -0.016779  
5 -0.001792  0.027190  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Total Returns   R-squared:                       0.656
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     785.3
Date:                Fri, 16 Jan 2026   Prob (F-statistic):          1.08e-285
Time:                        19:58:13   Log-Likelihood:                 5602.9
No. Observations:                1240   AIC:                        -1.120e+04
Df Residuals:                    1236   BIC:                        -1.118e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.784e-05   7.51e-05     -0.504      0.614      -0.000       0.000
PC2           -0.0016   6.71e-05    -23.276      0.000      -0.002      -0.001
PC3           -0.0031   7.45e-05    -41.609      0.000      -0.003      -0.003
PC4            0.0008   9.05e-05      9.102      0.000       0.001       0.001
==============================================================================
Omnibus:                       60.909   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              185.115
Skew:                          -0.148   Prob(JB):                     6.35e-41
Kurtosis:                       4.869   Cond. No.                         1.35
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(PortfCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPX_Idx","MOVE Idx","Vix Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Spread_Diff","ReturnSPX_Idx","MOVE Idx","Vix Idx"]))

X_pca = pcav1.transform(X_Std)

PortfCorrData.loc[:,"PC1"]=X_pca[:,0]
PortfCorrData.loc[:,"PC2"]=X_pca[:,1]
PortfCorrData.loc[:,"PC3"]=X_pca[:,2]
PortfCorrData.loc[:,"PC4"]=X_pca[:,3]
PortfCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(PortfCorrData[["PC1","PC2","PC3","PC4"]])
Y = PortfCorrData["Total Returns"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.39710152 0.20817572 0.16926846 0.11476007 0.07211318 0.03858105]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_OAS_Spread_Diff  ReturnSPX_Idx  \
0           0.580726     -0.157053                  0.566303      -0.488494   
1          -0.181059      0.462693                 -0.118005      -0.045967   
2          -0.011910      0.765364                  0.018127      -0.421714   
3          -0.081876     -0.162931                 -0.066477       0.129808   
4           0.180154      0.360509                  0.556027       0.725494   
5           0.768553      0.137671                 -0.592842       0.195515   

   MOVE Idx   Vix Idx  
0  0.114107  0.256408  
1  0.625364  0.588229  
2 -0.464834 -0.140830  
3 -0.614773  0.753349  
4 -0.043362 -0.013781  
5 -0.001426  0.026137  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Total Returns   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     605.1
Date:                Fri, 16 Jan 2026   Prob (F-statistic):          3.94e-289
Time:                        20:00:49   Log-Likelihood:                 5614.3
No. Observations:                1240   AIC:                        -1.122e+04
Df Residuals:                    1235   BIC:                        -1.119e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.784e-05   7.44e-05     -0.509      0.611      -0.000       0.000
PC1        -6.629e-05   4.82e-05     -1.375      0.169      -0.000    2.83e-05
PC2           -0.0015   6.66e-05    -23.255      0.000      -0.002      -0.001
PC3           -0.0031   7.38e-05    -42.345      0.000      -0.003      -0.003
PC4            0.0008   8.97e-05      9.206      0.000       0.001       0.001
==============================================================================
Omnibus:                       58.496   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              173.727
Skew:                          -0.141   Prob(JB):                     1.89e-38
Kurtosis:                       4.812   Cond. No.                         1.86
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(PortfCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPX_Idx","MOVE Idx","Vix Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Spread_Diff","ReturnSPX_Idx","MOVE Idx","Vix Idx"]))

X_pca = pcav1.transform(X_Std)

PortfCorrData.loc[:,"PC1"]=X_pca[:,0]
PortfCorrData.loc[:,"PC2"]=X_pca[:,1]
PortfCorrData.loc[:,"PC3"]=X_pca[:,2]
PortfCorrData.loc[:,"PC4"]=X_pca[:,3]
PortfCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(PortfCorrData[["PC2","PC3","PC4"]])
Y = PortfCorrData["Total Returns"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.39710152 0.20817572 0.16926846 0.11476007 0.07211318 0.03858105]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_OAS_Spread_Diff  ReturnSPX_Idx  \
0           0.580726     -0.157053                  0.566303      -0.488494   
1          -0.181059      0.462693                 -0.118005      -0.045967   
2          -0.011910      0.765364                  0.018127      -0.421714   
3          -0.081876     -0.162931                 -0.066477       0.129808   
4           0.180154      0.360509                  0.556027       0.725494   
5           0.768553      0.137671                 -0.592842       0.195515   

   MOVE Idx   Vix Idx  
0  0.114107  0.256408  
1  0.625364  0.588229  
2 -0.464834 -0.140830  
3 -0.614773  0.753349  
4 -0.043362 -0.013781  
5 -0.001426  0.026137  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Total Returns   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     805.6
Date:                Fri, 16 Jan 2026   Prob (F-statistic):          3.29e-290
Time:                        20:01:06   Log-Likelihood:                 5613.3
No. Observations:                1240   AIC:                        -1.122e+04
Df Residuals:                    1236   BIC:                        -1.120e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.784e-05   7.44e-05     -0.508      0.611      -0.000       0.000
PC2           -0.0015   6.66e-05    -23.247      0.000      -0.002      -0.001
PC3           -0.0031   7.39e-05    -42.330      0.000      -0.003      -0.003
PC4            0.0008   8.97e-05      9.203      0.000       0.001       0.001
==============================================================================
Omnibus:                       61.344   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              185.126
Skew:                          -0.157   Prob(JB):                     6.32e-41
Kurtosis:                       4.867   Cond. No.                         1.35
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(PortfCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPX_Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_Z_Spread_Diff","ReturnSPX_Idx"]))

X_pca = pcav1.transform(X_Std)

PortfCorrData.loc[:,"PC1"]=X_pca[:,0]
PortfCorrData.loc[:,"PC2"]=X_pca[:,1]
PortfCorrData.loc[:,"PC3"]=X_pca[:,2]
PortfCorrData.loc[:,"PC4"]=X_pca[:,3]
#PortfCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(PortfCorrData[["PC2","PC3","PC4"]])
Y = PortfCorrData["Total Returns"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.57091853 0.26282895 0.10799706 0.05825545]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_Z_Spread_Diff  ReturnSPX_Idx
0           0.608162     -0.195779                0.586678      -0.497613
1          -0.052146      0.898860               -0.014838      -0.434869
2           0.177882      0.369994                0.553623       0.724544
3           0.771866      0.129715               -0.590839       0.195719


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Total Returns   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.790
Method:                 Least Squares   F-statistic:                     1556.
Date:                Fri, 16 Jan 2026   Prob (F-statistic):               0.00
Time:                        20:01:43   Log-Likelihood:                 5911.0
No. Observations:                1240   AIC:                        -1.181e+04
Df Residuals:                    1236   BIC:                        -1.179e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.784e-05   5.85e-05     -0.646      0.518      -0.000     7.7e-05
PC2           -0.0036   5.71e-05    -62.300      0.000      -0.004      -0.003
PC3           -0.0024   8.91e-05    -27.494      0.000      -0.003      -0.002
PC4            0.0007      0.000      5.510      0.000       0.000       0.001
==============================================================================
Omnibus:                       41.860   Durbin-Watson:                   2.220
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              108.542
Skew:                          -0.036   Prob(JB):                     2.69e-24
Kurtosis:                       4.448   Cond. No.                         2.12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [3]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(PortfCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPX_Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Spread_Diff","ReturnSPX_Idx"]))

X_pca = pcav1.transform(X_Std)

PortfCorrData.loc[:,"PC1"]=X_pca[:,0]
PortfCorrData.loc[:,"PC2"]=X_pca[:,1]
PortfCorrData.loc[:,"PC3"]=X_pca[:,2]
PortfCorrData.loc[:,"PC4"]=X_pca[:,3]
#PortfCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(PortfCorrData[["PC2","PC3","PC4"]])
Y = PortfCorrData["Total Returns"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.57072755 0.2627524  0.10853401 0.05798605]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_OAS_Spread_Diff  ReturnSPX_Idx
0           0.608157     -0.188887                  0.586572      -0.500400
1          -0.062851      0.903828                 -0.004840      -0.423228
2           0.175979      0.357094                  0.555387       0.730110
3           0.771509      0.141072                 -0.589453       0.193436


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Total Returns   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.790
Method:                 Least Squares   F-statistic:                     1557.
Date:                Fri, 16 Jan 2026   Prob (F-statistic):               0.00
Time:                        21:58:27   Log-Likelihood:                 5911.2
No. Observations:                1240   AIC:                        -1.181e+04
Df Residuals:                    1236   BIC:                        -1.179e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.784e-05   5.85e-05     -0.646      0.518      -0.000     7.7e-05
PC2           -0.0036   5.71e-05    -62.590      0.000      -0.004      -0.003
PC3           -0.0024   8.88e-05    -26.943      0.000      -0.003      -0.002
PC4            0.0006      0.000      5.151      0.000       0.000       0.001
==============================================================================
Omnibus:                       44.511   Durbin-Watson:                   2.220
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              116.882
Skew:                          -0.075   Prob(JB):                     4.16e-26
Kurtosis:                       4.496   Cond. No.                         2.13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(PortfCorrData[["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Diff","ReturnSPX_Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["HY_Index_OAS_Diff","10yYieldDiff","IG_Index_OAS_Spread_Diff","ReturnSPX_Idx"]))

X_pca = pcav1.transform(X_Std)

PortfCorrData.loc[:,"PC1"]=X_pca[:,0]
PortfCorrData.loc[:,"PC2"]=X_pca[:,1]
PortfCorrData.loc[:,"PC3"]=X_pca[:,2]
PortfCorrData.loc[:,"PC4"]=X_pca[:,3]
#PortfCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(PortfCorrData[["PC1","PC2","PC3","PC4"]])
Y = PortfCorrData["Total Returns"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.57072755 0.2627524  0.10853401 0.05798605]
   HY_Index_OAS_Diff  10yYieldDiff  IG_Index_OAS_Spread_Diff  ReturnSPX_Idx
0           0.608157     -0.188887                  0.586572      -0.500400
1          -0.062851      0.903828                 -0.004840      -0.423228
2           0.175979      0.357094                  0.555387       0.730110
3           0.771509      0.141072                 -0.589453       0.193436


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Total Returns   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.790
Method:                 Least Squares   F-statistic:                     1167.
Date:                Fri, 16 Jan 2026   Prob (F-statistic):               0.00
Time:                        21:58:30   Log-Likelihood:                 5911.2
No. Observations:                1240   AIC:                        -1.181e+04
Df Residuals:                    1235   BIC:                        -1.179e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.784e-05   5.86e-05     -0.646      0.518      -0.000     7.7e-05
PC1         9.918e-06   3.88e-05      0.256      0.798   -6.61e-05     8.6e-05
PC2           -0.0036   5.71e-05    -62.566      0.000      -0.004      -0.003
PC3           -0.0024   8.89e-05    -26.932      0.000      -0.003      -0.002
PC4            0.0006      0.000      5.149      0.000       0.000       0.001
==============================================================================
Omnibus:                       44.527   Durbin-Watson:                   2.221
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              116.682
Skew:                          -0.078   Prob(JB):                     4.60e-26
Kurtosis:                       4.495   Cond. No.                         3.14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""